In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [2]:
# Khởi tạo hàm dùng để thu thập dữ liệu từ một url nào đó sử dụng thư viện requests để thu thập và bs4 để parse dữ liệu thu được. Đầu ra của hàm là một giá trị dictonary.
def get_content(url):
   page = requests.get(url, headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'})
   return BeautifulSoup(page.text, 'html.parser')

In [3]:
# TThu thập đường dẫn sản phẩm
fpt_products = []
for i in range(1,10):
    response = requests.get(f'https://fptshop.com.vn/apiFPTShop/Product/GetProductList?brandAscii=&url=https:%2F%2Ffptshop.com.vn%2Fmay-tinh-xach-tay%3Ftrang%3D{i}')
    json_data = response.json()
    for j in range(len(json_data['datas']['filterModel']['listDefault']['list'])):
        fpt_products.append({'name': json_data['datas']['filterModel']['listDefault']['list'][j]['name'], 'url': json_data['datas']['filterModel']['listDefault']['list'][j]['nameAscii'], 'price': json_data['datas']['filterModel']['listDefault']['list'][j]['productVariant']['price'] })

In [4]:
# Thu thập thông số kĩ thuật
for product in tqdm(fpt_products):
    response = requests.get(f"https://fptshop.com.vn/apiFPTShop/Product/GetProductDetail?name={product['url']}")
    json_data = response.json()
    for attribute in json_data['datas']['model']['product']['productAttributes']:
        product[attribute['attributeName']] = attribute['specName']

100%|██████████| 224/224 [04:02<00:00,  1.08s/it]


In [5]:
df = pd.DataFrame(data=fpt_products)

In [6]:
df

,name,url,price,Tốc độ CPU,CPU,Tốc độ BUS,Loại CPU,Tốc độ tối đa,Hãng CPU,Bộ nhớ đệm,Tên đầy đủ (CPU),Số luồng,Số lõi,RAM,Loại RAM,Số khe cắm rời,Tốc độ RAM,Dung lượng nâng cấp tối đa,Số RAM onboard,Tên đầy đủ (RAM),Số khe RAM còn lại,Màn hình cảm ứng,Công nghệ màn hình,Màn hình,Độ phân giải,Tần số quét,Loại cảm ứng,Tấm nền,Tên đầy đủ (Màn hình),Độ tương phản,Độ sáng,Tỷ lệ màn hình,Độ phủ màu,Kiểu màn hình,Card đồ họa,Card rời,Có Card rời,Có Card onbroad,Hãng (Card onbroad),Model (Card onbroad),...,Số cổng USB 3.0,Số cổng Thunderbolt,Số cổng Card reader,Senser,Công nghệ âm thanh,Số Loa,Nhận diện khuôn mặt,Mở khóa vân tay (Finger Print),Mật khẩu,Ổ đĩa quang,Loại ổ đĩa quang,Bàn phím số,Công nghệ đèn bàn phím,Kiểu bàn phím,TouchPad,Loại đèn bàn phím,Màu đèn LED,PIN có thể tháo rời,Loại PIN,PIN (Cell),Power Supply,Type,Version,Hệ điều hành,Chất liệu mặt bên ngoài cùng,Chất liệu khung màn hình,Width (mm),Tản nhiệt,Height (mm),Bản lề (Hinge / Kickstand),Trọng lượng (kg),Chất liệu mặt lưng,Kích thước (mm),Chất liệu mặt bàn phím + kê tay,Model Series,PN,Thời gian bảo hành,Xuất xứ,Năm ra mắt,Phụ kiện kèm theo (bút / bàn phím/ cáp chuyển…)
0,Asus TUF Gaming FX506LI-HN039T/i5-10300H,asus-tuf-fx506li-hn039t-i5-10300h,22790000.0,2.50 GHz,Core i5,Đang cập nhật,10300H,5.00 GHz,Intel,8 MB Smart Cache,Intel Core i5-10300H,8,4,8 GB,DDR4,2,2933 MHz,32 GB,0,8 GB DDR4 2933 MHz,1,Không,LED Backlit,"15.6""",1920 x 1080 Pixel,144 Hz,Không,IPS,"15.6"" Anti-glare LED-backlit",Đang cập nhật,250 nits,16:09,45% NTSC,LED,NVIDIA GeForce GTX1650Ti 4Gb,GeForce Series,Có,Có,Intel,630,...,1,0,0,Không,DTS Studio Sound,2,Không,Không,Có,Không,Không,Có,SRGB,Backlit Chiclet Keyboard,Hỗ trợ cử chỉ đa cảm ứng,LED,Đang cập nhật,Không,Lithium-ion,3 Cell,150 W,64 bit,Home Single Language,Windows 10,ABS plastic,ABS plastic,359,2 quạt,24.7,Bản lề đôi,2.3,Kim loại,256,ABS plastic,Asus TUF Gaming,90NR03T1-M02050,24 Tháng,Trung Quốc,2020,"sạc, sách HDSD"
1,Asus TUF Gaming FX505DT-HN478T/R7-3750H,asus-tuf-fx505dt-hn478t-r7-3750h,19590000.0,2.30 GHz,Ryzen 7,Đang cập nhật,3750H,4.00 GHz,AMD,4 MB L3 Cache,AMD Ryzen 7-3750H,8,4,8 GB,DDR4,2,2400 MHz,32 GB,0,8 GB DDR4 2400 MHz,1,Không,Anti-glare LED-backlit,"15.6""",1920 x 1080 Pixel,144 Hz,Không,IPS,"15.6"" Anti-glare LED-backlit",Đang cập nhật,250 nits,16:09,45% NTSC,LED,GeForce GTX 1650 4GB,GeForce Series,Có,Có,AMD,Đang cập nhật,...,1,0,0,Không,ASUS SonicMaster stereo,2,Không,Không,Có,Không,Không,Có,Đơn sắc,Backlit Chiclet Keyboard,Đang cập nhật,Đang cập nhật,Đang cập nhật,Không,Lithium-Polymer,3 Cell,150 W,64 bit,Đang cập nhật,Windows 10,Nhựa,Nhựa,360,2 quạt,26,Ergolift,2.2,Nhựa,262,Nhựa,Asus TUF Gaming,FX505DT-HN478T,24 Tháng,Trung Quốc,2020,"sạc, sách HDSD"
2,Acer Aspire Gaming A715-41G-R150/R7-3750H,acer-aspire-gaming-a715-41g-r150-r7-3750h,19190000.0,2.30 GHz,Ryzen 7,8 GT/s,3750H,4.00 GHz,AMD,4 MB L3 Cache,AMD Ryzen 7-3750H,8,4,8 GB,DDR4,2,2666 MHz,32 GB,0,8 GB DDR4 2666 MHz,1,Không,LED-backlit,"15.6""",1920 x 1080 Pixel,60 Hz,Không,IPS,"15.6"" LED-backlit",Đang cập nhật,Đang cập nhật,16:09,Đang cập nhật,LED,GeForce GTX 1650 Ti 4GB,GeForce Series,Có,Có,AMD,Vega 10,...,2,Đang cập nhật,Đang cập nhật,Không,Acer TrueHarmony,2,Không,Không,Có,Không,Không,Có,Đơn sắc,Đang cập nhật,Đang cập nhật,LED,Red,Không,Lithium-ion,3 Cell,Đang cập nhật,64 bit,Home,Windows 10,Nhựa,Nhựa,363.4,2 quạt,23.25,Đang cập nhật,2.1,Nhựa,254.5,Nhựa,Acer Aspire,NH.Q8SSV.004,12 Tháng,Trung Quốc,2020,"sạc, sách HDSD"
3,MSI Bravo 15 A4DCR-070VN/R5 4600H,msi-bravo-15-a4dcr-070vn-r5-4600h,19490000.0,3.00 GHz,Ryzen 5,8 GT/s,4600H,4.00 GHz,AMD,8 MB L3 Cache,AMD Ryzen 5-4600H,12,6,8 GB,DDR4,2,3200 MHz,64 GB,0,8 GB DDR4 3200 MHz,1,Không,LED-backlit,"15.6""",1920 x 1080 Pixel,60 Hz,Không,IPS,"15.6"" LED-backlit",Đang cập nhật,Đang cập nhật,16:09,45% NTSC,LED,Radeon RX 5300M 3GB,Radeon Series,Có,Có,AMD,Radeon,...,2,0,0,Không,Đang cập nhật,2,Không,Không,Có,Không,Không,Không,Đơn sắc,Chiclet Keyboard,Đang cập nhật,LED,Red,Không,Lithium-Polymer,3 Cell,180 W,64 bit,Home,W

In [7]:
# Lưu dữ liệu vào file ‘fpt_product.csv’
df.to_csv('fpt_product.csv', index = False, header = True)